In [ ]:
import gymnasium as gym
import torch
from skrl.utils import set_seed
from preprocess import preprocess
from transformers import DecisionTransformerConfig, DecisionTransformerModel

set_seed(42)

gym.register(
    id="MultiDatasetDiscretedTradingEnv",
    entry_point="mc_env:MultiDatasetDiscretedTradingEnv",
    disable_env_checker=True,
)

[skrl:INFO] Seed: 42


In [ ]:
env_cfg = dict(
    id="MultiDatasetDiscretedTradingEnv",
    dataset_dir="./data/train/month_15m/**/**/*.pkl",
    preprocess=preprocess,
    positions=[-1, 1],
    multiplier=[1, 2, 5],
    trading_fees=0.01,
    borrow_interest_rate=0.03,
    portfolio_initial_value=1e4,
    max_episode_duration=1000,
    verbose=0,
    window_size=60,
)

In [ ]:
configuration = DecisionTransformerConfig()
model = DecisionTransformerModel(configuration)
configuration = model.config

In [ ]:
env = gym.make(**env_cfg)
# env = gym.make_vec(
#     vectorization_mode="sync",
#     wrappers=[gym.wrappers.FlattenObservation],
#     num_envs=32,
#     **env_cfg,
# )

[skrl:INFO] Environment wrapper: gymnasium


In [ ]:

model = DecisionTransformerModel
# evaluation
model = model.to(device)
model.eval()

state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

state = env.reset()
states = torch.from_numpy(state).reshape(1, 1, state_dim).to(device=device, dtype=torch.float32)
actions = torch.zeros((1, 1, act_dim), device=device, dtype=torch.float32)
rewards = torch.zeros(1, 1, device=device, dtype=torch.float32)
target_return = torch.tensor(TARGET_RETURN, dtype=torch.float32).reshape(1, 1)
timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
attention_mask = torch.zeros(1, 1, device=device, dtype=torch.float32)

# forward pass
with torch.no_grad():
    state_preds, action_preds, return_preds = model(
        states=states,
        actions=actions,
        rewards=rewards,
        returns_to_go=target_return,
        timesteps=timesteps,
        attention_mask=attention_mask,
        return_dict=False,
    )

  0%|          | 32758/30000000 [02:40<18:01:22, 461.87it/s] 